# Chapter 3 — Parallelization (Google ADK)

Run multiple research sub-agents **in parallel** using `ParallelAgent`, then merge results with a `SequentialAgent`.

This demonstrates the ADK agent file structure (`agent.py` pattern) and parallel execution.

In [ ]:
import os
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

print("Google API Key set:", bool(os.environ.get("GOOGLE_API_KEY")))

## Define Parallel Research Agents

Each agent researches a different aspect of the same topic concurrently.

In [ ]:
from google.adk.agents import Agent, ParallelAgent, SequentialAgent

# Research agent 1: Technical aspects
tech_researcher = Agent(
    name="tech_researcher",
    model="gemini-2.0-flash",
    description="Researches technical aspects of a topic.",
    instruction="Research the technical details and implementation aspects of the given topic. Store your findings in the session state key 'tech_findings'.",
    output_key="tech_findings",
)

# Research agent 2: Market/business aspects
market_researcher = Agent(
    name="market_researcher",
    model="gemini-2.0-flash",
    description="Researches market and business aspects of a topic.",
    instruction="Research the market trends, business implications, and industry impact of the given topic. Store your findings in the session state key 'market_findings'.",
    output_key="market_findings",
)

# Research agent 3: Social/ethical aspects
social_researcher = Agent(
    name="social_researcher",
    model="gemini-2.0-flash",
    description="Researches social and ethical aspects of a topic.",
    instruction="Research the social impact, ethical considerations, and public perception of the given topic. Store your findings in the session state key 'social_findings'.",
    output_key="social_findings",
)

print("Research agents defined: tech, market, social")

## Create the Parallel Agent

All three researchers run simultaneously.

In [ ]:
parallel_research = ParallelAgent(
    name="parallel_research",
    sub_agents=[tech_researcher, market_researcher, social_researcher],
)

print("ParallelAgent created with", len(parallel_research.sub_agents), "sub-agents")

## Create the Merger Agent

After parallel research completes, a synthesizer merges all findings into a cohesive report.

In [ ]:
synthesizer = Agent(
    name="synthesizer",
    model="gemini-2.0-flash",
    description="Synthesizes research findings into a cohesive report.",
    instruction="""You are a research synthesizer. Combine the findings from all research agents into
a well-structured comprehensive report. Use the session state keys:
- 'tech_findings' for technical research
- 'market_findings' for market research  
- 'social_findings' for social/ethical research

Create a unified report with clear sections.""",
)

print("Synthesizer agent ready")

## Combine with SequentialAgent

First run parallel research, then synthesize.

In [ ]:
pipeline = SequentialAgent(
    name="research_pipeline",
    sub_agents=[parallel_research, synthesizer],
)

print("Sequential pipeline: parallel_research → synthesizer")

In [ ]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

session_service = InMemorySessionService()
runner = Runner(agent=pipeline, app_name="parallel_demo", session_service=session_service)

session = await session_service.create_session(app_name="parallel_demo", user_id="user1")

request = types.Content(
    role="user",
    parts=[types.Part(text="Research the current state of autonomous vehicles.")]
)

response = await runner.run_async(user_id="user1", session_id=session.id, new_message=request)

for event in response:
    if event.content and event.content.parts:
        print(f"[{event.author}]: {event.content.parts[0].text[:200]}...")
        print("---")

## Key Takeaways

- **`ParallelAgent`**: Runs sub-agents concurrently for faster execution
- **`SequentialAgent`**: Chains stages together (parallel research → synthesis)
- **`output_key`**: Each agent stores results in session state for downstream agents to consume
- **Pattern**: Fan-out (parallel) → Fan-in (synthesize)